## get Cause efffect phrase from the sentence.

In [1]:
import ast
import pandas as pd
#'data/in/semeval/semeval_causalityDetection.csv'
#'data/in/SCITE/SCITE_TrueFalse.csv'
#'data/in/RAG/ADE/ADE_causalityTEST.csv'
#load semeval data with embed

#'data/in/semeval/sem_CE_test.csv'
#'data/in/RAG/ADE/ADE_CE_extract_TEST.csv')
#'data/in/RAG/c_e_Extraction/SCITE_CE_extract_INPUT.csv'


pm_DB = pd.read_csv('data/in/KNN_RAG/ADE_CE_extract_TEST.csv')
instance_emb = pm_DB['embedings'].to_list()
instance_emb = [ast.literal_eval(item) for item in instance_emb]
instances = pm_DB['sentence'].to_list()
len(instances)

2271

In [2]:
#load fewshotDB with embed
semeval_data = pd.read_csv('data/in/KNN_RAG/comb_SCITEsemADE_CausalityPattern.csv')
fewshot_emb = semeval_data['embedings'].to_list()
fewshot_emb = [ast.literal_eval(item) for item in fewshot_emb]

examples = semeval_data['sentence'].to_list()
len(examples)

2365

In [3]:
import groq
from groq import Groq
client = groq.Groq(api_key= "enter your API key here..")#my key

#gemma2-9b-it# llama3-8b-8192

In [4]:
def C_E_CP_Identifier(causality_sentence, fewshot_examples):

    chat_completion = client.chat.completions.create(
    #
    # Required parameters
    #
    messages=[
        # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
          "role": "system",
          "content": '''
  
          You will be given a sentence that shows a causality relation. 
          Your task is to identify the cause phrase, effect phrase, and causality phrase from
          that sentence. Below is the hints and rules for the task.

          Hints: In sentences showing a causality relation, you can identify a pair of phrases
          where one phrase causes the other (effect). There will also be a phrase between the
          cause and effect phrases that indicates the causality relation; this is the 
          causality phrase. And in most sentences causality phrase can be seen between the 
          cause and effect phrase.
           

          Your output should be in JSON format with keys cause, effect, and causality_phrase:
          {"cause": "", "effect": "", "causality_phrase": ""}.
          
          Rule 1: If you are not able to find the cause phrase, effect phrase, and causality phrase 
          from the given sentence you can output empty string for the keys in the json format 
          and do not guess the answer. 
          
          Rule 2: Ensure that there are no overlapping phrases between the cause phrase, effect phrase,
          and causality phrase. For instance, consider the input sentence "Fever is caused by virus." 
          Your identified output without overlap should be:
          {"cause": "virus", "effect": "fever", "causality_phrase": "caused by"}
          Your output should not contain overlap, as shown below:
          {"cause": "by virus", "effect": "fever is caused", "causality_phrase": "caused by"}
          This means that each phrase (cause, effect, and causality phrase) should be distinct 
          and not overlap with each other.
                   
          Below are 10 example sentences that show causality relations, with their cause and
          effect phrases delimited between the XML tags <cause>, </cause>, <effect>, and 
          </effect>. The corresponding causality phrases are also provided.
                   
          ''' + fewshot_examples
        },
        {
          "role": "user",
          "content": input_sentence 
        },
    ],

    # The language model which will generate the completion.
    model="gemma2-9b-it",
    temperature=0.0,
    seed = 1,
    max_tokens=100,
    top_p=1,
    response_format={"type": "json_object"},)
    out = chat_completion.choices[0].message.content
    
    return out

In [5]:
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
filtered_rows = pm_DB

cause_list = []
effect_list = []
causality_phrase_list = []
fewshotfound_inputsentence_index=[]
for i in range (len(filtered_rows)):
    input_sentence = filtered_rows['sentence'][i]
    #input_sentence = 'fever is caused by virus'
    
    #print('input_sentence', input_sentence)
    
    # few shot examples augmentation.
    ##########KNN###########
    # Combine lists into a numpy array
    lists = np.array(fewshot_emb)

    # Calculate cosine similarity between list_1 and other lists
    similarities = cosine_similarity([instance_emb[i]], fewshot_emb).flatten()

    # Get indices of the two most similar lists
    most_similar_indices = similarities.argsort()[-10:][::-1]

    # Print the most similar lists and their cosine similarity values

    fewshot_examples = [examples[j] for j in most_similar_indices]

    fewshot_examples = {f'example {k + 1}': sentence for k, sentence in enumerate(fewshot_examples)}
    fewshot_examples = json.dumps(fewshot_examples)   

    #####################
    
    # output generation
    try:
        out = C_E_CP_Identifier(input_sentence, fewshot_examples)

        print('out: ', out)

        json_out = json.loads(out)
        cause = json_out['cause']
        effect = json_out['effect']
        causality_phrases = json_out['causality_phrase']
        
    except TypeError as e:
        print('TypeError')
        cause_list.append('')
        effect_list.append('')
        causality_phrase_list.append('')
        continue
        
    except KeyError as e:
        print('KeyError')
        cause_list.append('')
        effect_list.append('')
        causality_phrase_list.append('')
        continue
    
    except groq.APIConnectionError as e:
        print('APIConnectionError')
        cause_list.append('')
        effect_list.append('')
        causality_phrase_list.append('')
        continue
        
    except groq.RateLimitError as e:
        print('RateLimitError')
        cause_list.append('')
        effect_list.append('')
        causality_phrase_list.append('')       
        continue
        
    except groq.APIStatusError as e:
        print('APIStatusError')
        cause_list.append('')
        effect_list.append('')
        causality_phrase_list.append('')
        continue
    

    
    cause_list.append(cause)
    effect_list.append(effect)
    causality_phrase_list.append(causality_phrases)
    
    #print('input_sentence: ', input_sentence )
    #print('fewshot_examples: ', fewshot_examples )
    #print('cause: ', cause)
    #print('effect: ', effect)
    #print('causality_phrase: ', causality_phrases)
    
    
    print(i)
    #if i==0:
        #break;
    #print(causality_identifier)
    
    

out:  {"cause": "5-fluorouracil", "effect": "Squamous-cell carcinoma arising in a basal-cell epithelioma", "causality_phrase": "treated with"}
0
out:  {"cause": "long-term therapy with mitomycin C and 5-fluorouracil", "effect": "acute renal toxicity", "causality_phrase": "receiving"}
1
out:  {"cause": "budesonide", "effect": "contact dermatitis", "causality_phrase": "causing"}
2
out:  {"cause": "mannitol-induced ARF", "effect": "rapid reversal", "causality_phrase": "for rapid reversal of"}
3
out:  {"cause": "discontinuation of chlorambucil", "effect": "clinical AML", "causality_phrase": "After"}
4
out:  {"cause": "L-asparaginase therapy", "effect": "seizures and altered sensorium", "causality_phrase": "after"}
5
out:  {"cause": "a regimen of carbamazepine, lynestrenol and sodium valproate", "effect": "hepatopathy", "causality_phrase": "during"}
6
out:  {"cause": "anastrozole use", "effect": "Diagnosis of sclerosing glomerulonephritis", "causality_phrase": "suggesting a newly defined si

out:  {"cause": "clindamycin and trimethoprim-sulfamethoxazole", "effect": "Cholestatic liver disease with ductopenia (vanishing bile duct syndrome)", "causality_phrase": "after administration of"}
64
out:  {"cause": "L-dopa treatment", "effect": "rapid mood cycling", "causality_phrase": "during"}
65
out:  {"cause": "a small initial dose of prazosin ranging from 0.5 to 1 mg", "effect": "avoid the first-dose phenomenon characterized by a sudden and severe drop in blood pressure after the administration of the first dose of prazosin", "causality_phrase": "to avoid"}
66
out:  {"cause": "chlorpropamide", "effect": "Acute intravascular hemolysis", "causality_phrase": "was started on"}
67
out:  {"cause": "a higher dose than recommended was used, there was recent or concomitant treatment with mefloquine, there was pre-existing prolongation of the QT interval or the patient had a thiamine deficiency", "effect": "8 cardiac arrests, leading to 6 deaths", "causality_phrase": "leading to"}
68
out:

out:  {"cause": "sulindac", "effect": "incapacitating isolated idential sensory neuropathy", "causality_phrase": "exposed to"}
122
out:  {"cause": "defective calcium-activated K+ channels", "effect": "rhabdomyolysis", "causality_phrase": "cause"}
123
out:  {"cause": "rifabutin therapy", "effect": "uveitis", "causality_phrase": "associated with"}
124
out:  {"cause": "sulfadiazine", "effect": "crystalluria and renal failure", "causality_phrase": "induced"}
125
out:  {"cause": "isotretinoin (1 mg/kg/day) for cystic acne", "effect": "poor night vision and/or excessive glare sensitivity", "causality_phrase": "treated with"}
126
out:  {"cause": "calcineurin-induced pain syndrome (CIPS)", "effect": "pain syndrome", "causality_phrase": "induced"}
127
out:  {"cause": "olanzapine", "effect": "rapidly occurring hyperglycemia", "causality_phrase": "treatment with"}
128
out:  {"cause": "Magnesium tocolysis", "effect": "urinary calculus during pregnancy", "causality_phrase": "as the cause of"}
129
o

out:  {"cause": "Phenylpropanolamine", "effect": "psychosis", "causality_phrase": "induced"}
184
out:  {"cause": "Barbiturate", "effect": "submassive hepatic necrosis", "causality_phrase": "induced"}
185
out:  {"cause": "quinine-associated thrombotic microangiopathy", "effect": "TTP patients with ADAMTS13 was measured before plasma exchange", "causality_phrase": "for whom"}
186
out:  {"cause": "IFN therapy", "effect": "Graves' hyperthyroidism", "causality_phrase": "during"}
187
out:  {"cause": "bevacizumab", "effect": "spontaneous nasal septal perforation", "causality_phrase": "after given"}
188
out:  {"cause": "anastrozole", "effect": "renal injury", "causality_phrase": "due to"}
189
out:  {"cause": "extemporaneous topical minoxidil", "effect": "myocardial infarction", "causality_phrase": "had hypertension and arteriosclerotic disease and the other died of"}
190
out:  {"cause": "metoclopramide", "effect": "Reversible nonthrombocytopenic palpable purpura", "causality_phrase": "associat

out:  {"cause": "hypersensitivity reactions to cyclosporine", "effect": "future cyclosporine use", "causality_phrase": "investigate the mechanisms involved in"}
247
out:  {"cause": "methotrexate", "effect": "CNS abnormalities", "causality_phrase": "induced"}
248
out:  {"cause": "L-thyroxine therapy", "effect": "acute myocardial infarction", "causality_phrase": "due to"}
249
out:  {"cause": "a single dose of vincristine", "effect": "severe visual loss", "causality_phrase": "following"}
250
out:  {"cause": "penicillamine", "effect": "TTP in RA", "causality_phrase": "induced"}
251
out:  {"cause": "starting citalopram", "effect": "episodes of chest tightness and dizziness", "causality_phrase": "after starting"}
252
out:  {"cause": "DCF", "effect": "keratitis of moderate severity", "causality_phrase": "ocular toxicity of"}
253
out:  {"cause": "glibenclamide therapy", "effect": "pemphigus vulgaris", "causality_phrase": "precipitated by"}
254
out:  {"cause": "fluvoxamine-induced akathisia", "

...

out:  {"cause": "initiating treatment with interferon alfa", "effect": "onset of anterior ischemic optic neuropathy", "causality_phrase": "between"}
2001
out:  {"cause": "i.v. pulse methylprednisolone (IVPMP) treatment", "effect": "seizures and transient blindness", "causality_phrase": "after"}
2002
out:  {"cause": "phosphate-induced hypokalemia", "effect": "potassium loss", "causality_phrase": "occurred by"}
2003
out:  {"cause": "sustained-release procainamide", "effect": "granulocytopenia", "causality_phrase": "caused by"}
2004
out:  {"cause": "use of acyclovir", "effect": "syndrome of fever, pulmonary infiltrates, and pleural effusion", "causality_phrase": "following use of"}
2005
out:  {"cause": "treatment with DCF", "effect": "immunosuppression and superinfection", "causality_phrase": "of treatment with"}
2006
out:  {"cause": "the administration of the cephalosporin", "effect": "the coagulopathy", "causality_phrase": "seemed to be most closely associated with"}
2007
out:  {"cause"

out:  {"cause": "aspirin prophylaxis", "effect": "hemorrhage from a meningioma", "causality_phrase": "may have been induced by"}
2065
out:  {"cause": "chlorambucil therapy", "effect": "EEG abnormalities", "causality_phrase": "during"}
2066
out:  {"cause": "thrombin", "effect": "cerebral infarctions", "causality_phrase": "may have been responsible for"}
2067
out:  {"cause": "fludarabine", "effect": "severe AIHA in CLL patients", "causality_phrase": "treated with"}
2068
out:  {"cause": "a regimen of methotrexate for 18 years", "effect": "Progressive interstitial fibrosis with roentgenographic honeycombing", "causality_phrase": "developed in the case of"}
2069
out:  {"cause": "cytarabine", "effect": "cerebellar syndrome", "causality_phrase": "cytarabine-induced"}
2070
out:  {"cause": "pentazocine toxicity", "effect": "irreversible drug-induced fibrous myopathy and localized neuropathy", "causality_phrase": "in order to prevent"}
2071
out:  {"cause": "Improved awareness of and further inve

out:  {"cause": "27 months of D-penicillamine therapy and a cumulative dose of 1,200 g", "effect": "rapidly progressive glomerulonephritis with epithelial crescents associated with hemoptysis", "causality_phrase": "after"}
2126
out:  {"cause": "thiopurine methyltransferase deficiency", "effect": "myelosuppression", "causality_phrase": "due to"}
2127
out:  {"cause": "cyclosporine", "effect": "hypersensitivity reactions", "causality_phrase": "to"}
2128
out:  {"cause": "Hydroxyurea", "effect": "concomitant occurrence of diffuse longitudinal melanonychia and multiple squamous cell carcinomas", "causality_phrase": "associated with"}
2129
out:  {"cause": "ampicillin sodium challenge", "effect": "severe temporary weakness", "causality_phrase": "after"}
2130
out:  {"cause": "Propoxyphene", "effect": "wide complex dysrhythmia", "causality_phrase": "induced"}
2131
out:  {"cause": "lamotrigine administration", "effect": "TEN and SJS", "causality_phrase": "related to"}
2132
out:  {"cause": "5-FU-i

out:  {"cause": "sulfasalazine", "effect": "lupus", "causality_phrase": "induced"}
2185
out:  {"cause": "radiation fibrosis", "effect": "occurrence of ILD in patients taking erlotinib", "causality_phrase": "may be another contributor of"}
2186
out:  {"cause": "Efalizumab and Etanercept", "effect": "a case of tuberculosis", "causality_phrase": "for treatment of"}
2187
out:  {"cause": "three weeks of carbamazepine therapy", "effect": "severe agitation and aggressive behavior", "causality_phrase": "After"}
2188
out:  {"cause": "methysergide maleate", "effect": "migraine headaches", "causality_phrase": "received for"}
2189
out:  {"cause": "rifampicin", "effect": "clinical evidence of adrenal failure", "causality_phrase": "treated with"}
2190
out:  {"cause": "Treosulfan for ovarian cancer", "effect": "acute leukemia", "causality_phrase": "treated with"}
2191
out:  {"cause": "switched from stavudine to zidovudine", "effect": "relapse of symptomatic hyperlactatemia", "causality_phrase": "afte

out:  {"cause": "high-dose fluticasone propionate", "effect": "adrenal suppression", "causality_phrase": "after treatment with"}
2248
out:  {"cause": "prednisone therapy", "effect": "Severe hyperkalemia", "causality_phrase": "shortly after beginning"}
2249
out:  {"cause": "lithium", "effect": "syndrome of irreversible lithium-effectuated neurotoxicity (SILENT)", "causality_phrase": "effectuated"}
2250
out:  {"cause": "switching to zidovudine", "effect": "hyperlactatemia relapse", "causality_phrase": "potentially can lead to"}
2251
out:  {"cause": "adalimumab", "effect": "Acute bilateral phrenic neuropathy", "causality_phrase": "following treatment with"}
2252
out:  {"cause": "Unintended exposure to acyclovir early in pregnancy", "effect": "excessive maternal and physician anxiety", "causality_phrase": "may cause"}
2253
out:  {"cause": "MTX administration", "effect": "Epstein-Barr virus-associated polymorphic lymphoproliferative disorder (LPD)", "causality_phrase": "due to immunodeficie

In [6]:
len(filtered_rows)

2271

In [7]:
#filtered_rows = filtered_rows.drop('GPT_causality_label', axis = 1)
#filtered_rows = filtered_rows.drop('patternmatcher_label', axis = 1)
len(fewshotfound_inputsentence_index)

0

In [8]:
filtered_rows['cause'] = cause_list
filtered_rows['effect'] = effect_list
filtered_rows['causality_phrase'] = causality_phrase_list
filtered_rows

,Unnamed: 0.1,Unnamed: 0,sentence,cause_t,effect_t,embedings,cause,effect,causality_phrase
0,0,0,Squamous-cell carcinoma arising in a basal-cel...,5-fluorouracil,Squamous-cell carcinoma,"[-0.010072928853332996, 0.0002307597897015512,...",5-fluorouracil,Squamous-cell carcinoma arising in a basal-cel...,treated with
1,1,1,Three cases of acute renal toxicity in patient...,5-fluorouracil,acute renal toxicity,"[0.003660476766526699, -0.006745004560798407, ...",long-term therapy with mitomycin C and 5-fluor...,acute renal toxicity,receiving
2,2,2,CONCLUSIONS: Although budesonide may be benefi...,budesonide,contact dermatitis,"[-0.016381533816456795, 0.014267788268625736, ...",budesonide,contact dermatitis,causing
3,3,3,Hemodialysis should be performed for rapid rev...,mannitol,ARF,"[-0.009250188246369362, 0.008307494223117828, ...",mannitol-induced ARF,rapid reversal,for rapid reversal of
4,4,4,"After the chlorambucil was discontinued, the w...",chlorambucil,AML,"[0.0037177789490669966, -0.024612924084067345,...",discontinuation of chlorambucil,clinical AML,After
...,...,...,...,...,...,...,...,...,...
2266,2266,2266,We report here on a heretofore undescribed res...,fludarabine phosphate,respiratory syncytial virus (RSV) infection,"[-0.012739364989101887, -0.0047148303128778934...",fludarabine phosphate,respiratory syncytial virus (RSV) infection,treated with
2267,2267,2267,We describe the first documented case of azath...,azathioprine,myelosuppression,"[-0.0030996142886579037, -0.015832170844078064...",thiopurine methyltransferase deficiency,severe myelosuppression,due to
2268,2268,2268,We report a case of biopsy-proven acute tubulo...,flurbiprofen,acute tubulointerstitial nephritis,"[0.004250906873494387, -0.0017411339795216918,...",a second course of flurbiprofen,biopsy-proven acute tubulointerstitial nephritis,associated with
2269,2269,2269,Onset of male gynaecomastia in a patient treat...,sunitinib,gynaecomastia,"[-0.024055609479546547, -0.02433030679821968, ...",sunitinib,onset of male gynaecomastia,treated with


In [9]:
def get_score(true_phrase, pred_phrase ):
    scores=[]
    p = true_phrase
    window_size = len(p)

        # Slide through input_sentence with a window size of 8
    if (pred_phrase) =="":
        print('no pred_phrase')
        return 0
    else: 
        for i in range(len(pred_phrase) - window_size + 1):
            window = pred_phrase[i:i + window_size]
            score = fuzz.ratio(window, p)
            scores.append(score)
        if len(scores) ==0:
            print('prediction length lower', pred_phrase, 'p:', p)
            return 0
        else:
            return max(scores)

In [10]:
#get accuracy
from fuzzywuzzy import fuzz

def get_accuracy(filtered_rows):
    prediction= []
    e_prediction=[]
    c_prediction=[]
    for i in range(len(filtered_rows)):
        #print(i)
        cause_pred = filtered_rows['cause'][i]
        cause_t = filtered_rows['cause_t'][i]
        
        effect_pred = filtered_rows['effect'][i]
        effect_t = filtered_rows['effect_t'][i]

        # Slide through input_sentence with a window size of 8
        #print('effect_pred',  effect_pred)
        #print('effect_t',  effect_t)


        score = get_score(cause_t.lower(), cause_pred.lower())################ non -medcause
        #score = fuzz.ratio(cause_pred, cause_t)################medcause
        if score >90:################90 non -medcause
            pred_c = 1
            c_prediction.append(pred_c)
        else:
            pred_c = 0
            c_prediction.append(pred_c)
            
        score = get_score(effect_t.lower(), effect_pred.lower())################non - medcause
        #score = fuzz.ratio(effect_pred, effect_t)################medcause

        if score >90: ################90 non -medcause
            pred_e = 1
            e_prediction.append(pred_e)
        else:
            pred_e = 0
            e_prediction.append(pred_e)

        
        if ((pred_e ==1) & (pred_c ==  1)):
            prediction.append(1)
        else:
            prediction.append(0)
            
    

    return prediction, c_prediction, e_prediction


In [11]:
prediction, c_prediction, e_prediction = get_accuracy(filtered_rows)

no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
prediction length lower chronic renal failure p: acute hemorrhagic gastritis
prediction length lower hypercalcemia p: vitamin d intoxication
no pred_phrase
no pred_phrase
prediction length lower methylphenidate p: dextroamphetamine
no pred_phrase
no pred_phrase
prediction length lower aortic wall was damaged p: aortic wall in this case was damaged
prediction length lower serum lithium levels p: increase serum lithium levels
prediction length lower this observation p: electrophysiologic effect
prediction length lower myocardial infarction p: arteriosclerotic disease
prediction length lower theophylline intoxication p: decrease in theophylline clearance
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
prediction length lower unconsciousness p: elevation of serum creatinine and blood urea
prediction length lower future cyclosporine use p: hypersensitivity reactions
no pred_phrase
no pred_phrase
prediction length lower 

In [12]:
sum(prediction)/len(filtered_rows)

0.8256274768824307

In [13]:
sum(c_prediction)/len(filtered_rows)

0.9119330691325407

In [14]:
sum(e_prediction)/len(filtered_rows)

0.8789079700572435

In [15]:
filtered_rows['predictions'] = prediction

In [16]:
filtered_rows

,Unnamed: 0.1,Unnamed: 0,sentence,cause_t,effect_t,embedings,cause,effect,causality_phrase,predictions
0,0,0,Squamous-cell carcinoma arising in a basal-cel...,5-fluorouracil,Squamous-cell carcinoma,"[-0.010072928853332996, 0.0002307597897015512,...",5-fluorouracil,Squamous-cell carcinoma arising in a basal-cel...,treated with,1
1,1,1,Three cases of acute renal toxicity in patient...,5-fluorouracil,acute renal toxicity,"[0.003660476766526699, -0.006745004560798407, ...",long-term therapy with mitomycin C and 5-fluor...,acute renal toxicity,receiving,1
2,2,2,CONCLUSIONS: Although budesonide may be benefi...,budesonide,contact dermatitis,"[-0.016381533816456795, 0.014267788268625736, ...",budesonide,contact dermatitis,causing,1
3,3,3,Hemodialysis should be performed for rapid rev...,mannitol,ARF,"[-0.009250188246369362, 0.008307494223117828, ...",mannitol-induced ARF,rapid reversal,for rapid reversal of,0
4,4,4,"After the chlorambucil was discontinued, the w...",chlorambucil,AML,"[0.0037177789490669966, -0.024612924084067345,...",discontinuation of chlorambucil,clinical AML,After,1
...,...,...,...,...,...,...,...,...,...,...
2266,2266,2266,We report here on a heretofore undescribed res...,fludarabine phosphate,respiratory syncytial virus (RSV) infection,"[-0.012739364989101887, -0.0047148303128778934...",fludarabine phosphate,respiratory syncytial virus (RSV) infection,treated with,1
2267,2267,2267,We describe the first documented case of azath...,azathioprine,myelosuppression,"[-0.0030996142886579037, -0.015832170844078064...",thiopurine methyltransferase deficiency,severe myelosuppression,due to,0
2268,2268,2268,We report a case of biopsy-proven acute tubulo...,flurbiprofen,acute tubulointerstitial nephritis,"[0.004250906873494387, -0.0017411339795216918,...",a second course of flurbiprofen,biopsy-proven acute tubulointerstitial nephritis,associated with,1
2269,2269,2269,Onset of male gynaecomastia in a patient treat...,sunitinib,gynaecomastia,"[-0.024055609479546547, -0.02433030679821968, ...",sunitinib,onset of male gynaecomastia,treated with,1


In [17]:
#filtered_rows.to_csv('data/out/RAG_C_E_CP_Alzeimers.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/semeval/fewshot_sem_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_

#filtered_rows.to_csv('data/out/RAG_causality/CE_extracts/randomshotCOMB_ADE_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/task2/gemma/KNN_ADE_CE_extract.csv')#zero,rand,RAG|sem,ADE,SCITE


In [18]:
#filtered_rows= filtered_rows[filtered_rows['predictions'] == 0]
#filtered_rows.to_csv('data/out/RAG_causality/semeval/zero_sem_RAG_C_E_pred0.csv')#data/out/RAG_causality/semeval/semeval_
